In [1]:
from aicsimageio import AICSImage
import napari
from aicsimageio.readers import CziReader
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [55]:
# FUNCTION DEFINITIONS


# VISUALIZATION

# visualize the image with napari using its numpy array
def visualize_napari(numpy_img: np.ndarray,name):
    """
    :param numpy_img: image to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(numpy_img,name=name)


# visualize multiple images at once
def visualize_all_list_napari(numpy_img_list: np.ndarray,names):
    """
    :param numpy_img_list: list containing different images to be visualized
    """
    with napari.gui_qt():
        viewer = napari.Viewer()
        for i, img in enumerate(numpy_img_list):
            viewer.add_image(img, name=names[i] )




# PREPROCESSING

# sharpen image
def sharpen(image: np.ndarray):
    """
    Sharpen the image
    :param image: image to be sharpened
    :return: sharp image
    """
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    return cv.filter2D(image, -1, kernel)




# THRESHOLDING

# compute Otsu's thresholding
def otsu_thresholding(image : np.ndarray):
    """
    Threshold and binarize an image using Otsu's method

    :param image: image you want to threshold
    :return: ret: threshold value
              th: binary image
    """
    ret,th = cv.threshold(image,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    # ret is the computed value of the threshold AND th is the image with the threshold applied
    return ret, th


# split the whole images into tiles
def split_into_tiles(image : np.ndarray, tile_size: int):
    """
    split image into tiles of shape tile_size*tile_size

    :param image: image to be split
    :param tile_size: dimensions of single tiles
    :return: tiles: list with the different tiles
    """
    tiles = []
    
    for i in range(0, image.shape[0], tile_size):
        for j in range(0, image.shape[1], tile_size):
            tile = image[i:i+tile_size, j:j+tile_size]
            tiles.append(tile)
    
    return tiles


# reconstruct image from different tiles given the number of tiles in x and y direction and a list of tiles
def reconstruct_image(tiles: list, x_tiles: int, y_tiles: int):
    """
    :param tiles:    list with the different single tiles
    :param x_tiles:  number of tiles along x axis
    :param y_tiles:  number of tiles along y axis
    :return:         numpy array, reconstructed image
    """
    big_image = np.zeros((x_tiles*tiles[0].shape[0], y_tiles*tiles[0].shape[1]))
    
    for i in range(x_tiles):
        for j in range(y_tiles):
            big_image[i*tiles[0].shape[0]:(i+1)*tiles[0].shape[0], j*tiles[0].shape[1]:(j+1)*tiles[0].shape[1]] = tiles[i*y_tiles+j]
    
    return big_image


# apply Otsu tresholding on tiles after sharpening and splitting image
def otsu_split_thresholding(img: np.ndarray, tile_size = 16):
    """
    Perform Otsu tresholding on sub images of 16 x 16,
    if a tile is all white do not apply otsu

    :param img:  input image
    :return thresholded_tiles_sharp: list with thresholded tiles, to be recomposed
    """
    # sharpen image
    sharpened_img = sharpen(img)
    
    # get the maximum of the sharpened img, needed to check if image is all white
    max_value=sharpened_img.max()
    
    # split
    tiles_sharpened = split_into_tiles(sharpened_img, tile_size)
    
    # do thresholding
    thresholded_tiles_sharp=[]
    
    for t in tiles_sharpened:
        # check if mostly white #ATTENTION, im touching the og in memory. If yes set direclty to black
        if check_all_white_tile(t,max_value):
            th = set_zero(t)
            thresholded_tiles_sharp.append(th)
        # else do thresholding
        else:
            r,th = otsu_thresholding(t)
            thresholded_tiles_sharp.append(th)
    
    return thresholded_tiles_sharp


# clean the result of Otsu's thresholding on tiles
def otsu_cleaned_split_thresholding(img):
    """
    Perform otsu thresholding on 16 x 16 images, then clean the image, 
    delete the noise

    :param img: image to be tresholded
    :return:    tresholded clean image
    """
    # list with the thresholded tiles size 16x16
    thresholded_tiles = otsu_split_thresholding(img,16)

    # clean
    cleaned_tiles=[]
    for tl in thresholded_tiles:
        # check if image is not a bacilli
        if check_image(tl):
            # I'm not a bacilli
            m = set_zero(tl)
            cleaned_tiles.append(m)
        else:
            # I am a bacilli
            cleaned_tiles.append(tl)

    #reconstruct
    reconstructed_clean_image = reconstruct_image(cleaned_tiles,128,94)

    #final cleaning
    #final_cleaned_image = (reconstructed_clean_image)

    #visualize
    #visualize_all_list_napari([reconstructed_image, reconstructed_clean_image,sharpened_img,img], ["reconstructed_image","reconstructed_clean_image","sharpened_img","img"])

    return reconstructed_clean_image



#POSTPROCESSING

# Check whether or not to keep specific tiles of image
def check_image(img: np.ndarray):
    """
    For every sub-image we check if its worth keeping or not
    215 pretty hard-coded---> maybe rely on scientific paper to find the optimal number

    :param img: image to be checked
    :return: bool
    """
    if np.sum(img == 0) > 215: #we have a bacilli
        return False
    else:
        return True


# set pixels that are 255 to zero (black)
def set_zero(img):
    h = img
    h[h>0] = 0
    return h


def check_all_white_tile(img,max_value_global):
    """
    Check if we have a huge bright tile. if a 16 x 16 tile is all white--->
    we want it black. Check based on global max pixel value

    :param img: tile to be checked if white
    :param max_value: max value pixel of whole image
    :return:
    """
    if np.sum(img > 0.2* max_value_global) > 0.8*img.shape[0]*img.shape[1]:
        return True
    else:
        return False


def clean_connected_components(img: np.ndarray):
    """
    Clean image with 2 approaches: delete connected components that have are up to 2 pixels
                                   connect bacilli that are separated by just one black pixel

    :param img: image to be cleaned
    :return:    cleaned image
    """
    # find connected components
    num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(np.uint8(img), connectivity=8)
    # stats = x,y,w,h,area

    # put to black connected components which area is equal to 1 or 2
    for i in range(1, num_labels):
        if stats[i][4] < 3:
            img[labels_im == i] = 0

    # do not want to connect bacilli in original, want to connect after little components are gone
    img2 = img.copy()
    
    # connect the bacilli, by putting a white tile
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            if img[i,j] == 0:
                if (img[i-1,j] == 255 and img[i+1,j] == 255) or (img[i,j-1] == 255 and img[i,j+1] == 255) \
                        or (img[i-1,j-1] ==255 and img[i+1,j+1]) or (img[i-1,j+1] == 255 and img[i+1,j-1] == 255) \
                        or (img[i-1,j]== 255 and img[i+1,j+1]==255) or (img[i-1,j+1]==255 and img[i+1,j]==255)\
                        or (img[i-1,j]==255 and img[i+1,j-1]==255) or (img[i-1,j-1]==255 and img[i+1,j]==255)\
                        or (img[i,j-1]==255 and img[i+1,j+1]==255) or (img[i,j-1]==255 and img[i-1,j+1]==255)\
                        or (img[i,j+1]==255 and img[i+1,j-1]==255) or (img[i,j+1]==255 and img[i-1,j-1]==255):
                    img2[i,j] = 255

    return img2


# add 2d bounding boxes to the image
def add_bounding_boxes(image, stats):
    """
    Add white rectangles around bacilli, based on conected components

    :param image: image with bacilli to be boxed
    :param coordinates:  coordinates of the center of the bacillus
    """
    for i in range(1,len(stats)):
            x = stats[i][0] - 5
            #x_max = coordinates[i][0]
            y = stats[i][1] - 5
            #y_max = coordinates[i][1]
            h=stats[i][3]
            w=stats[i][2]
            cv.rectangle(image, (x, y), (x+w+10, y+h+10), (255, 0, 0), 1)
    
    return image

In [3]:
#PIPELINE (until the end)

# Load the image
reader = CziReader("extern_Synlab_2156_17_3_MTB.czi")

# Get whole image
smear = reader.get_image_data("MYX", C=0)

In [45]:
# save in a new variable the information regarding the 673th tile out of 1345
img = smear[674]
img.shape
real_image = img.copy()

In [46]:
# for comparison
_, otsu_thresholded_entire_og_img = otsu_thresholding(img)
sharpened_img = sharpen(img)
_, otsu_thresholded_entire_sharp_img = otsu_thresholding(sharpened_img)


In [47]:
# do split thresholding
otsu_st_16 = reconstruct_image(otsu_split_thresholding(img,16),128,94)

In [48]:
# do split thresholding and clean noise from thresholding
otsu_st_16_cleaned_from_noise = otsu_cleaned_split_thresholding(img)

In [49]:
# save new copy for comparison, not needed if we dont want comparison
a_bit_of_shit_in_our_bacilli_tiles = otsu_st_16_cleaned_from_noise.copy()

In [50]:
# cut small connected components and connect bacilli
cleaning_the_shit_on_the_bacilli_tiles = clean_connected_components(otsu_st_16_cleaned_from_noise)

In [51]:
# get stats for drawing boxes
num_labels, labels_im, stats, centroids = cv.connectedComponentsWithStats(np.uint8(cleaning_the_shit_on_the_bacilli_tiles), connectivity=8)

In [52]:
# another copy----ask marina
bacilli = otsu_st_16_cleaned_from_noise.copy()

In [58]:
# add the boxes
box_bacilli = add_bounding_boxes(bacilli, stats)

# add boxes to original image
img_copy = img.copy()
box_bacilli_og = add_bounding_boxes(img_copy, stats)

In [59]:
# visualize
visualize_all_list_napari([box_bacilli,box_bacilli_og,cleaning_the_shit_on_the_bacilli_tiles,a_bit_of_shit_in_our_bacilli_tiles,otsu_st_16_cleaned_from_noise,otsu_st_16,img],["boxes","boxes in og","no 1s and unite bacilli", "a bit of artifacts","cleaned split otsu","split otsu","og"])

c:\Users\idabu\.vscode\TBProject\.venv\lib\site-packages\napari\_qt\qt_event_loop.py:284: FutureWarning: 
The 'gui_qt()' context manager is deprecated.
If you are running napari from a script, please use 'napari.run()' as follows:

    import napari

    viewer = napari.Viewer()  # no prior setup needed
    # other code using the viewer...
    napari.run()

In IPython or Jupyter, 'napari.run()' is not necessary. napari will automatically
start an interactive event loop for you: 

    import napari
    viewer = napari.Viewer()  # that's it!

  warn(


9852870
